In [1]:
from pprint import pprint
import numpy as np
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from pyvi import ViTokenizer

In [2]:
""" Load data
"""
train_X_path = "./data/telesale/processed_100_calls/texts.txt"
train_y_path = "./data/telesale/processed_100_calls/labels.txt"
val_X_path = "./data/telesale/validation/texts.txt"
val_y_path = "./data/telesale/validation/labels.txt"

def read_file(file):
    contents = []
    with open(file, "r") as f:
        lines = f.readlines()
        for line in lines:
            contents.append(line.strip())
    return contents

train_X = read_file(train_X_path)
train_y = read_file(train_y_path)
val_X = read_file(val_X_path)
val_y = read_file(val_y_path)

In [3]:
""" Feature Extraction
"""
def _tokenize(text):
    return ViTokenizer.tokenize(text).split()

vectorizer = TfidfVectorizer(
    token_pattern=r"(?u)\b\w+\b",
    tokenizer=_tokenize,
    ngram_range=(1, 2),
    sublinear_tf=True
)

train_X = vectorizer.fit_transform(train_X)
val_X = vectorizer.transform(val_X)

In [4]:
""" Prepare multilabel format for training with Sklearn
"""
# Get unique labels from dataset
# for label binarizer
classes = set()
for label in train_y:
    tmp_lbl = label.split(",")
    for t in tmp_lbl:
        classes.add(t)
classes = list(classes)
mlb = MultiLabelBinarizer(classes=classes)

# Binarize label
train_y = mlb.fit_transform([tuple(l.split(",")) for l in train_y])
val_y = mlb.transform([tuple(l.split(",")) for l in val_y])

In [5]:
""" Build classifier which enables multilabel capability
"""
RANDOM_STATE = 42
NUM_CPU = 8
estimator = SVC(kernel='linear',
                random_state=RANDOM_STATE,
                verbose=1, probability=True)
clf = MultiOutputClassifier(estimator, n_jobs=NUM_CPU)
params = {"estimator__C": [0.001, 0.01, 0.1, 1, 10]}
gs = GridSearchCV(
    estimator=clf,
    param_grid=params,
    scoring="f1_macro",
    n_jobs=NUM_CPU,
    cv=5,
    verbose=3,
    error_score=0.,
    return_train_score=True
)
gs.fit(train_X, train_y)
model = gs.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
*
optimization finished, #iter = 1
obj = -0.001999, rho = 0.999656
nSV = 2, nBSV = 2
Total nSV = 2
*
optimization finished, #iter = 1
obj = -0.001999, rho = 0.999638
nSV = 2, nBSV = 2
Total nSV = 2
*
optimization finished, #iter = 1
obj = -0.001999, rho = 0.999654
nSV = 2, nBSV = 2
Total nSV = 2
*
optimization finished, #iter = 1
obj = -0.001999, rho = 0.999656
nSV = 2, nBSV = 2
Total nSV = 2
*
optimization finished, #iter = 2
obj = -0.001999, rho = -0.999595
nSV = 3, nBSV = 1
Total nSV = 3
*
optimization finished, #iter = 1
obj = -0.001999, rho = 0.999673
nSV = 2, nBSV = 2
Total nSV = 2
**
optimization finished, #iter = 1

optimization finished, #iter = 1
obj = -0.001999, rho = 0.999668
nSV = 2, nBSV = 2
Total nSV = 2
obj = -0.001999, rho = 0.999623
nSV = 2, nBSV = 2
Total nSV = 2
*
optimization finished, #iter = 1
obj = -0.001999, rho = 0.999666
nSV = 2, nBSV = 2
Total nSV = 2
**
optimization finished, #iter = 1
obj = -0.001

/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to 0.0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/multioutput.py", line 434, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/multioutput.py", line 206, in

*
optimization finished, #iter = 48
obj = -2.409665, rho = 1.074934
nSV = 31, nBSV = 0
Total nSV = 31
*
optimization finished, #iter = 52
obj = -2.395860, rho = 1.059194
nSV = 37, nBSV = 0
Total nSV = 37
*
optimization finished, #iter = 102
obj = -2.177626, rho = 1.024486
nSV = 78, nBSV = 0
Total nSV = 78
*
optimization finished, #iter = 136
obj = -8.358275, rho = 0.990496
nSV = 91, nBSV = 0
Total nSV = 91
*
optimization finished, #iter = 111
obj = -4.831982, rho = 1.055290
nSV = 72, nBSV = 0
Total nSV = 72
*
optimization finished, #iter = 154
obj = -4.315455, rho = 1.035453
nSV = 107, nBSV = 0
Total nSV = 107
*
optimization finished, #iter = 225
obj = -9.376029, rho = 1.015977
nSV = 155, nBSV = 0
Total nSV = 155
*
optimization finished, #iter = 117
obj = -2.103441, rho = 0.992849
nSV = 93, nBSV = 0
Total nSV = 93
*
optimization finished, #iter = 402
obj = -65.498415, rho = 1.205583
nSV = 180, nBSV = 0
Total nSV = 180
*
optimization finished, #iter = 105
obj = -4.836485, rho = 1.054953

In [6]:
pprint(gs.cv_results_)
print(f"best estimator: {gs.best_estimator_}")
print(f"best params: {gs.best_params_}")
print(f"best score: {gs.best_score_}")

{'mean_fit_time': array([1.50522652, 2.36267853, 3.5447926 , 3.86501942, 2.86501908]),
 'mean_score_time': array([0.1265698 , 0.30186582, 0.40167918, 0.36301455, 0.15377274]),
 'mean_test_score': array([0.        , 0.        , 0.0118579 , 0.19655046, 0.21796312]),
 'mean_train_score': array([0.        , 0.        , 0.01320753, 0.55095739, 0.8       ]),
 'param_estimator__C': masked_array(data=[0.001, 0.01, 0.1, 1, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object),
 'params': [{'estimator__C': 0.001},
            {'estimator__C': 0.01},
            {'estimator__C': 0.1},
            {'estimator__C': 1},
            {'estimator__C': 10}],
 'rank_test_score': array([4, 4, 3, 2, 1], dtype=int32),
 'split0_test_score': array([0., 0., 0., 0., 0.]),
 'split0_train_score': array([0., 0., 0., 0., 0.]),
 'split1_test_score': array([0.        , 0.        , 0.0167754 , 0.24756179, 0.30314586]),
 'split1_train_score': array([0.        , 0. 

In [7]:
y_pred = model.predict(val_X)
y_true = val_y
f1_micro = f1_score(y_true, y_pred, average="micro")
f1_macro = f1_score(y_true, y_pred, average="macro")
accuracy = accuracy_score(y_true, y_pred)
print(f"f1 micro: {f1_micro:.2f}")
print(f"f1 macro: {f1_macro:.2f}")
print(f"accuracy: {accuracy:.2f}")

f1 micro: 0.99
f1 macro: 0.51
accuracy: 0.98


/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [8]:
y_pred = model.predict(train_X)
y_true = train_y
f1_micro = f1_score(y_true, y_pred, average="micro")
f1_macro = f1_score(y_true, y_pred, average="macro")
print(f"f1 micro: {f1_micro:.2f}")
print(f"f1 macro: {f1_macro:.2f}")

f1 micro: 1.00
f1 macro: 1.00


In [25]:
test_x = ["rồi sau hai tháng này ví ví dụ như á mà anh đóng tiền hai tháng đầu tiên á là anh sẽ đóng cùng này với cái hợp đồng cũ của anh khi nào hợp đồng cũ của mình hết đúng không mà khả năng anh có thì mình thanh lý cái gốc lại cho bên em là mình sẽ cộng thêm phí thanh lý là năm phần trăm trên tổng số tiền dư nợ gốc đầu kỳ còn nếu như mà sau hai tháng hợp đồng cũ của anh nó đã hết rồi và ở bên đây em chưa hoàn gốc lại được cho bên em á thì sang tháng thứ ba bên em sẽ chia nhỏ khoản vay này ra để hỗ trợ anh góp thì mỗi tháng anh góp lại giúp em số tiền là hai triệu sáu à lẻ chín ngàn thì trong cái khoản này anh góp nó sẽ có bao gồm cho mình như là gốc lãi bảo hiểm năm chấm năm với lại mười hai ngàn phí thu hộ cho mình và thời gian thì bên em sẽ chia cho mình là ba mươi bốn tháng ha ờ bây giờ á ví dụ như mà anh góp khoản tầm được một hai tháng mà anh có tiền á mình thanh lý bên em vẫn hỗ trợ nha bên em chỉ nhận cái dư nợ gốc còn lại của anh thôi và những tháng như về sau là mình không tính lãi cộng phí tất toán là năm phần trăm anh ha"]
test_x = vectorizer.transform(test_x)
predicted_results = model.predict(test_x)
print(predicted_results)
print(np.where(predicted_results == 1)[1])
# mlb.inverse_transform(predicted_results)

[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]]
[12]
